In [1]:
import csv
import chess
rows = []
with open('lichess_db_puzzle.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        rows.append(row)

#row[0] = PuzzleID
#row[1] = FEN
#row[2] = Moves
#row[3] = Rating
#row[4] = RatingDeviation
#row[5] = Popularity
#row[6] = NbPlayes
#row[7] = Themes
#row[8] = URL
#row[9] = OpeningTags
#row[10] = GoodRowThemes
#row[11] = CountGoodRowThemes
header = rows[0]
rows.remove(rows[0])
#print(f"{header[1]:<70}{header[2]:<40}{header[3]:<15}{header[5]:<15}{header[6]:<15}{header[7]:<30}")
#for row in rows:
#    print(f"{row[1]:<70}{row[2]:<40}{row[3]:<15}{row[5]:<15}{row[6]:<15}{row[7]:<30}")




In [2]:
final = []
for row in rows[1:]:
    if not(int(row[5]) < 70 or int(row[6]) < 1000): #int(row[3]) < 50 or int(row[3]) > 4000
        final.append(row)
print("number of puzzles: ",len(final))
    

number of puzzles:  1088967


In [3]:
final2 = []
for row in final:
    if not("superGM" in row[7] or "master" in row[7] or "arabianMate" in row[7] or "bodenMate" in row[7] or "doubleBishopMate" in row[7] or "hookMate" in row[7] or "oneMove" in row[7] or "quietMove" in row[7] or "underPromotion" in row[7] or "zugzwang" in row[7]):
        final2.append(row)

print("number of puzzles: ",len(final2))

number of puzzles:  913536


In [4]:
piece_names = {
    'P': 'Pawn',
    'N': 'Knight',
    'B': 'Bishop',
    'R': 'Rook',
    'Q': 'Queen',
    'K': 'King'
}
themes = set()
for row in final2:
    moves = row[2].split()
    board = chess.Board(row[1])
    used_pieces = set()
    for i in range(len(moves)):
        move = chess.Move.from_uci(moves[i])
        if i % 2 == 1:  # Check if the index is odd (every second move)
            piece = board.piece_at(move.from_square).symbol().upper()
            if piece not in used_pieces:  # Check if the piece is new
                used_pieces.add(piece)  # Add new piece to the set
                row[7] += " " + piece_names.get(piece, "Unknown")
        board.push(move)
    rowThemes = row[7].split()
    goodRowThemes = []
    badThemes = ['short','attackingF2F7','knightEndgame', 'pawnEndgame','mate','bishopEndgame','dovetailMate','queenEndgame','backRankMate','anastasiaMate','rookEndgame','veryLong','smotheredMate','long','queenRookEndgame']
    for theme in rowThemes:
        if (theme not in badThemes):
            themes.add(theme)
            goodRowThemes.append(theme)
    row.append(goodRowThemes)
    row.append(len(goodRowThemes))

print(themes)
print(len(themes))
  

{'mateIn3', 'advantage', 'advancedPawn', 'endgame', 'skewer', 'doubleCheck', 'middlegame', 'promotion', 'clearance', 'queensideAttack', 'xRayAttack', 'Bishop', 'intermezzo', 'mateIn5', 'exposedKing', 'kingsideAttack', 'opening', 'discoveredAttack', 'defensiveMove', 'sacrifice', 'fork', 'Rook', 'enPassant', 'castling', 'capturingDefender', 'pin', 'attraction', 'trappedPiece', 'Pawn', 'deflection', 'Knight', 'hangingPiece', 'interference', 'crushing', 'Queen', 'King', 'mateIn4', 'mateIn2'}
38


In [5]:
#Get New CSV File with all important Puzzles and the important Themes filtered out
header.append('GoodRowThemes')
header.append('CountGoodRowThemes')
final2.insert(0, header)
with open('ImportantPuzzles.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the data
    writer.writerows(final2)

In [6]:
import random
import xlsxwriter
import numpy as np

def getPossiblePuzzles(rating: int):
    possiblePuzzles = []
    for row in final2[1:]:
        if (int(row[3]) > rating -80 and int(row[3]) < rating +80):
            possiblePuzzles.append(row)
    
    return possiblePuzzles

def findLessThanX(selectedPuzzles2, timesToOccur):
    theme_count = {} #directory key-value pairs

    # Count the occurrences of each theme
    for puzzle in selectedPuzzles2:
        for theme in puzzle[10]:
            if theme in theme_count:
                theme_count[theme] += 1 #add 1 to count if already has a counter
            else:
                theme_count[theme] = 1 #add a counter and set to 1
    lessThanXCount = []
    for theme in themes:
        if(theme not in theme_count or theme_count[theme] < timesToOccur):
            lessThanXCount.append(theme)
    return lessThanXCount, theme_count

def ensure_themes_appear_X_times(initialPuzzles2, timesToOccur):

    lessThanX, theme_count = findLessThanX(initialPuzzles2, timesToOccur)

    # Add puzzles to the list, focusing on underrepresented themes
    while (len(lessThanX) > 0):
        randomID = random.randint(0,len(possiblePuzzles)-1)
        puzzle = possiblePuzzles[randomID]
        if (puzzle not in initialPuzzles2):
            for puzzleTheme in puzzle[10]:
                if puzzleTheme in lessThanX:
                    for puzzleTheme2 in puzzle[10]:
                        if theme in theme_count:
                            theme_count[theme] += 1 #add 1 to count if already has a counter
                        else:
                            theme_count[theme] = 1 #add a counter and set to 1
                    initialPuzzles2.append(puzzle)
                    lessThanX, theme_count = findLessThanX(initialPuzzles2, timesToOccur)
                    break


    return initialPuzzles2

def write_to_excel(initialIDs, filename):
    workbook = xlsxwriter.Workbook(filename)
    worksheet = workbook.add_worksheet()

    # Write the data
    for row_num, row_data in enumerate(initialIDs):
        for col_num, cell_data in enumerate(row_data):
            worksheet.write(row_num, col_num, cell_data)

    workbook.close()

for i in [2000]:  

    possiblePuzzles = getPossiblePuzzles(i)
    initialPuzzles = []
    timesToOccur = 15
    countXInitialPuzzles = ensure_themes_appear_X_times(initialPuzzles, timesToOccur)

    print("Anzahl Count", timesToOccur, "Puzzles:",len(countXInitialPuzzles), "Rating: ", i)

    randomShuffling = np.random.choice(range(0, len(countXInitialPuzzles)), size=len(countXInitialPuzzles), replace=False)

    countXInitialPuzzlesShuffled = [countXInitialPuzzles[i] for i in randomShuffling]

    initialIDs = []
    for puzzle in countXInitialPuzzlesShuffled:
        initialIDs.append (["=HYPERLINK(\"https://lichess.org/training/" + puzzle[0] + "\")", ""])

    #Get New CSV File with all initial Puzzles
    initialHeader = ["PuzzleLink","Richtig(1) oder Falsch(0)"]
    initialIDs.insert(0, initialHeader)

    countXInitialPuzzlesShuffled.insert(0, header)
    playerPuzzles = 'InitialPuzzles' + str(i) + '.csv'
    with open(playerPuzzles, 'w', newline='') as file:
        writer = csv.writer(file)

        # Write the data
        writer.writerows(countXInitialPuzzlesShuffled)

    playerIDs = 'InitialIDs' + str(i) + '.xlsx'
    write_to_excel(initialIDs, playerIDs)


Anzahl Count 15 Puzzles: 341 Rating:  2000


In [7]:
ratings = []
for i in range(1, len(final2)):
    ratings.append(int(final2[i][3]))

print(max(ratings))
print(min(ratings))

3244
399
